In [1]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv('../data/labs.csv', header=None)

In [5]:
data.head()

,0,1,2,3,4,5
0,1175090,2020-05-26,CKD-EPI,2.16,NaN,"ml/s/1,73 m2"
1,1175090,2024-05-07,CKD-EPI,1.89,NaN,"ml/s/1,73 m2"
2,1175090,2020-05-26,s_kreatinin,73.10,NaN,µmol/l
3,1175090,2024-05-07,s_kreatinin,84.30,NaN,µmol/l
4,1175090,2024-05-07,UACR,0.40,NaN,g/mol


In [6]:
import sqlite3
import csv

# Connect to SQLite database
conn = sqlite3.connect('../data/CKD_train.db')
cursor = conn.cursor()

# Create table
cursor.execute('''
CREATE TABLE IF NOT EXISTS patients (
    Patient INTEGER,
    DateOfBirth TEXT,
    Sex TEXT
)
''')

# Open the CSV file and read its contents
with open('../data/patients.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header row
    rows = [(int(row[1]), row[2], row[3]) for row in reader]

# Insert CSV data into the table
cursor.executemany('''
INSERT INTO patients (Patient, DateOfBirth, Sex)
VALUES (?, ?, ?)
''', rows)

# Commit the transaction and close the connection
conn.commit()
conn.close()

In [16]:
import sqlite3
import pandas as pd
import numpy as np

db_path = "../data/CKD_train.db"
conn = sqlite3.connect(db_path)

query_labs = "SELECT Patient, EntryDate, Analyte, ValueNumber, ValueText, Unit FROM labs"
query_patients = "SELECT Patient, Sex FROM patients"

df_labs = pd.read_sql_query(query_labs, conn)
df_patients = pd.read_sql_query(query_patients, conn)

conn.close()

df_labs["EntryDate"] = pd.to_datetime(df_labs["EntryDate"])

df_labs = df_labs.sort_values(by=["Patient", "EntryDate"])

output_data = []

latest_egfr = {}
latest_uacr = {}

def calculate_egfr(creatinine, sex, age):
    creatinine = creatinine / 88.4  # Conversion µmol/L -> mg/dL
    
    if sex == "F":
        if creatinine <= 0.7:
            egfr = 144 * (creatinine / 0.7) ** -0.329 * (0.993 ** age)
        else:
            egfr = 144 * (creatinine / 0.7) ** -1.209 * (0.993 ** age)
    else:  # Male
        if creatinine <= 0.9:
            egfr = 141 * (creatinine / 0.9) ** -0.411 * (0.993 ** age)
        else:
            egfr = 141 * (creatinine / 0.9) ** -1.209 * (0.993 ** age)
    
    return egfr

# Fonction pour attribuer un risque CKD en fonction de eGFR et UACR
def classify_ckd_risk(egfr, uacr):
    # Déterminer la catégorie eGFR (G1 à G5)
    if egfr >= 90:
        gfr_category = "G1"
    elif 60 <= egfr < 90:
        gfr_category = "G2"
    elif 45 <= egfr < 60:
        gfr_category = "G3a"
    elif 30 <= egfr < 45:
        gfr_category = "G3b"
    elif 15 <= egfr < 30:
        gfr_category = "G4"
    else:
        gfr_category = "G5"

    # Déterminer la catégorie UACR (A1 à A3)
    if uacr < 3:
        acr_category = "A1"
    elif 3 <= uacr < 30:
        acr_category = "A2"
    else:
        acr_category = "A3"

    # Déterminer le risque selon la matrice CKD
    risk_matrix = {
        ("G1", "A1"): 1, ("G1", "A2"): 2, ("G1", "A3"): 3,
        ("G2", "A1"): 1, ("G2", "A2"): 2, ("G2", "A3"): 3,
        ("G3a", "A1"): 2, ("G3a", "A2"): 3, ("G3a", "A3"): 4,
        ("G3b", "A1"): 3, ("G3b", "A2"): 4, ("G3b", "A3"): 4,
        ("G4", "A1"): 4, ("G4", "A2"): 4, ("G4", "A3"): 4,
        ("G5", "A1"): 4, ("G5", "A2"): 4, ("G5", "A3"): 4,
    }

    return risk_matrix.get((gfr_category, acr_category), np.nan)

# Fusionner avec les sexes des patients
df_labs = df_labs.merge(df_patients, on="Patient", how="left")

# Itérer sur chaque ligne
for _, row in df_labs.iterrows():
    patient_id = row["Patient"]
    entry_date = row["EntryDate"]
    analyte = row["Analyte"]
    value_number = row["ValueNumber"]
    value_text = row["ValueText"]
    sex = row["Sex"]  # Sexe du patient

    # Hypothèse : âge moyen de 50 ans en l'absence de données
    age = 50

    # Initialiser les valeurs
    egfr, uacr = np.nan, np.nan

    # Si l'analyse concerne la créatinine sérique
    if analyte == "s_kreatinin" and pd.notna(value_number):
        egfr = calculate_egfr(value_number, sex, age)
        latest_egfr[patient_id] = egfr

    # Si l'analyse concerne l'UACR
    elif analyte == "UACR":
        # Vérifier si la valeur est dans ValueNumber sinon mettre NaN
        if pd.notna(value_number):
            uacr = value_number*1000
        else: 
            uacr = np.nan
        latest_uacr[patient_id] = uacr

    # Obtenir les valeurs les plus récentes si absentes
    egfr = latest_egfr.get(patient_id, np.nan) if pd.isna(egfr) else egfr
    uacr = latest_uacr.get(patient_id, np.nan) if pd.isna(uacr) else uacr

    # Calculer le risque CKD
    risk = classify_ckd_risk(egfr, uacr)

    # Ajouter à la liste de résultats
    output_data.append([patient_id, entry_date, egfr, uacr, risk])

# Créer le DataFrame final
df_output = pd.DataFrame(output_data, columns=["Patient", "EntryDate", "eGFR", "UACR", "Risk"])


In [17]:
df_output['UACR'].unique()

array([       nan, 3.2000e+03, 2.4000e+03, 3.4000e+03, 1.5800e+04,
       7.3000e+03, 1.6000e+03, 2.3000e+03, 1.7000e+03, 2.9700e+04,
       3.0300e+04, 2.2600e+04, 2.7900e+04, 3.4100e+04, 4.1000e+04,
       5.6000e+04, 1.3960e+05, 1.5170e+05, 5.7700e+04, 9.0000e+02,
       6.0000e+02, 1.1000e+03, 1.2000e+03, 3.9000e+03, 2.2000e+03,
       1.4000e+03, 1.5000e+03, 4.7000e+03, 6.4000e+03, 1.1800e+04,
       3.5000e+03, 5.4000e+03, 1.0000e+03, 9.7000e+03, 7.6000e+03,
       7.8000e+03, 8.3000e+03, 1.3400e+04, 4.1000e+03, 1.5000e+04,
       4.3000e+03, 9.2000e+03, 8.7000e+03, 1.8900e+04, 2.0000e+04,
       1.2100e+04, 1.1500e+04, 1.6900e+04, 8.1000e+03, 8.5000e+03,
       1.2200e+04, 8.0000e+02, 5.9000e+03, 2.5000e+03, 4.0000e+03,
       1.1900e+04, 4.7000e+04, 3.5300e+04, 7.4100e+04, 8.2700e+04,
       1.5200e+05, 3.0000e+02, 4.0000e+02, 5.1200e+04, 9.5900e+04,
       7.4900e+04, 9.1800e+04, 1.2170e+05, 4.9820e+05, 1.0100e+04,
       1.2720e+05, 6.6000e+03, 1.0200e+04, 1.2130e+05, 1.7040e